In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/6.862_MachineLearningProject/BuildingEnergyConsumption/Code

/content/drive/MyDrive/6.862_MachineLearningProject/BuildingEnergyConsumption/Code


In [3]:
# Imports:
import numpy as np
import pandas as pd
import os
from os.path import join
import glob
import matplotlib.pyplot as plt
from load_and_process_data import process_data
from sklearn import preprocessing

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

### 1. Import all data into single pandas dataframe:

In [4]:
# In jupyter import files like:
#datafolder = join(os.getcwd()[:os.getcwd().index('Code')]) + 'Data/ashrae-energy-prediction/'
#pd_all = process_data(datafolder)

In [5]:
# In colab import files like:
datafolder = '../Data/ashrae-energy-prediction/'
pd_all = process_data(datafolder)

In [6]:
pd_all

,site_id,building_id,primary_use,square_feet,year_built,floor_count,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter,meter_reading
0,0,0,Education,7432,2008.0,5.0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.70,0.0,0.0,0,0.000
1,0,0,Education,7432,2008.0,5.0,2016-01-01 01:00:00,24.4,4.0,21.1,-1.0,1020.20,70.0,1.5,0,0.000
2,0,0,Education,7432,2008.0,5.0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.20,0.0,0.0,0,0.000
3,0,0,Education,7432,2008.0,5.0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.10,0.0,0.0,0,0.000
4,0,0,Education,7432,2008.0,5.0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.00,250.0,2.6,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,15,1448,Office,92271,2001.0,4.0,2016-12-31 19:00:00,3.0,1.0,-8.0,-1.0,1007.55,180.0,5.7,0,3.400
20125601,15,1448,Office,92271,2001.0,4.0,2016-12-31 20:00:00,2.8,2.0,-8.9,-1.0,1007.40,180.0,7.7,0,2.700
20125602,15,1448,Office,92271,2001.0,4.0,2016-12-31 21:00:00,2.8,2.0,-7.2,-1.0,1007.50,180.0,5.1,0,3.050
20125603,15,1448,Office,92271,2001.0,4.0,2016-12-31 22:00:00,2.2,2.0,-6.7,-1.0,1008.00,170.0,4.6,0,3.125


In [7]:
#newArrayWithOneHotEncoding = pd.get_dummies(arrayThatYouWantToTransform, columns = ['firstColumnHeader'])
pd.get_dummies(pd_all, columns = ['primary_use'], prefix='', prefix_sep='')

,site_id,building_id,square_feet,year_built,floor_count,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter,meter_reading,Education,Entertainment/public assembly,Food sales and service,Healthcare,Lodging/residential,Manufacturing/industrial,Office,Other,Parking,Public services,Religious worship,Retail,Services,Technology/science,Utility,Warehouse/storage
0,0,0,7432,2008.0,5.0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.70,0.0,0.0,0,0.000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,7432,2008.0,5.0,2016-01-01 01:00:00,24.4,4.0,21.1,-1.0,1020.20,70.0,1.5,0,0.000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,7432,2008.0,5.0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.20,0.0,0.0,0,0.000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,7432,2008.0,5.0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.10,0.0,0.0,0,0.000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,7432,2008.0,5.0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.00,250.0,2.6,0,0.000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,15,1448,92271,2001.0,4.0,2016-12-31 19:00:00,3.0,1.0,-8.0,-1.0,1007.55,180.0,5.7,0,3.400,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20125601,15,1448,92271,2001.0,4.0,2016-12-31 20:00:00,2.8,2.0,-8.9,-1.0,1007.40,180.0,7.7,0,2.700,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20125602,15,1448,92271,2001.0,4.0,2016-12-31 21:00:00,2.8,2.0,-7.2,-1.0,1007.50,180.0,5.1,0,3.050,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20125603,15,1448,92271,2001.0,4.0,2016-12-31 22:00:00,2.2,2.0,-6.7,-1.0,1008.00,170.0,4.6,0,3.125,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [8]:
dictionary_meter = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}
pd_all = pd_all.replace({'meter':dictionary_meter})
pd_all = pd.get_dummies(pd_all, columns = ['meter','primary_use'], prefix=['',''],prefix_sep=['',''])

In [9]:
list(pd_all)

['site_id',
 'building_id',
 'square_feet',
 'year_built',
 'floor_count',
 'timestamp',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'sea_level_pressure',
 'wind_direction',
 'wind_speed',
 'meter_reading',
 'chilledwater',
 'electricity',
 'hotwater',
 'steam',
 'Education',
 'Entertainment/public assembly',
 'Food sales and service',
 'Healthcare',
 'Lodging/residential',
 'Manufacturing/industrial',
 'Office',
 'Other',
 'Parking',
 'Public services',
 'Religious worship',
 'Retail',
 'Services',
 'Technology/science',
 'Utility',
 'Warehouse/storage']

In [10]:
bins = [0,1940, 1980, np.inf] 
names = ['1900-1940', '1940-1980', '1980-today']

pd_all['year_range'] = pd.cut(pd_all['year_built'], bins, labels=names)

In [11]:
pd_all = pd_all.drop(columns = 'year_built')

In [12]:
len(list(pd_all))

34

In [13]:
pd_all = pd.get_dummies(pd_all, columns = ['year_range'])

In [14]:
len(list(pd_all))

36

In [15]:
list(pd_all)

['site_id',
 'building_id',
 'square_feet',
 'floor_count',
 'timestamp',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'sea_level_pressure',
 'wind_direction',
 'wind_speed',
 'meter_reading',
 'chilledwater',
 'electricity',
 'hotwater',
 'steam',
 'Education',
 'Entertainment/public assembly',
 'Food sales and service',
 'Healthcare',
 'Lodging/residential',
 'Manufacturing/industrial',
 'Office',
 'Other',
 'Parking',
 'Public services',
 'Religious worship',
 'Retail',
 'Services',
 'Technology/science',
 'Utility',
 'Warehouse/storage',
 'year_range_1900-1940',
 'year_range_1940-1980',
 'year_range_1980-today']

In [16]:
pd_all = pd_all.drop(columns = 'site_id')

In [17]:
pd_all.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
building_id,20125605.0,797.211616,426.372886,0.0,391.0,894.00,1176.000,1448.0
square_feet,20125605.0,107784.793238,117161.626346,283.0,32527.0,72709.00,139113.000,875000.0
floor_count,20125605.0,4.108074,2.099450,1.0,2.0,4.00,5.000,26.0
air_temperature,20125605.0,15.987859,10.947337,-28.9,8.6,16.70,24.100,47.2
cloud_coverage,20125605.0,2.776449,2.710668,0.0,0.0,2.00,4.500,9.0
dew_temperature,20125605.0,7.747271,10.178279,-35.0,0.0,8.90,16.100,26.1
precip_depth_1_hr,20125605.0,1.181800,7.893033,-1.0,0.0,0.00,0.000,343.0
sea_level_pressure,20125605.0,1016.086378,6.916425,968.2,1011.8,1016.30,1020.300,1045.5
wind_direction,20125605.0,171.212374,112.653692,0.0,80.0,170.00,270.000,360.0
wind_speed,20125605.0,3.375851,2.264048,0.0,2.1,3.10,4.600,19.0


In [18]:
normalizer = preprocessing.Normalization()